# Geração de Texto usando LangChain e Hugging Face

##### 📜 **Descrição**
- Este projeto demonstra como utilizar modelos de linguagem (LLMs) da Hugging Face dentro do LangChain para geração de texto. O código utiliza o modelo `google/gemma-2b-it`, um LLM otimizado para tarefas de instrução (instruction-tuned) e realiza a geração de respostas a partir de entradas fornecidas pelo usuário.

- A implementação inclui quantização 4-bit para permitir a execução do modelo em GPUs com menos memória, garantindo mais eficiência no processamento.

- Este projeto pode ser usado como base para comparar o desempenho do Gemma com outros modelos hospedados na Hugging Face.


🔍 **Ficha Técnica**  

| 🔍 **Item** | 📄 **Descrição** |
|------------|----------------|
| **🛠️ Tecnologias** | Python, PyTorch, Hugging Face Transformers, LangChain |
| **📦 Dependências** | `transformers`, `torch`, `bitsandbytes`, `accelerate`, `langchain`, `langchain_huggingface` |
| **⚙️ Funcionalidade** | Geração de texto usando LLMs otimizados para GPU |
| **📌 Modelo Utilizado** | `google/gemma-2b-it` |
| **📉 Quantização** | 4-bit (`BitsAndBytesConfig`) para eficiência |
| **🔑 Autenticação** | **Requer token da Hugging Face** para acessar o modelo |

## Instalação e Importação de Bibliotecas

In [1]:
#!pip install -q bitsandbytes
!pip install -U bitsandbytes
!pip install -q torch transformers langchain langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 760.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 35.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
import os
import getpass

In [17]:
# Definir semente para reprodutibilidade

#torch.random.manual_seed(42)

## Definição do token
colar o token gerado no painel da Hugging Face

In [9]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


## Configuração do Modelo com Quantização

In [10]:
model_id = "google/gemma-2b-it" #o modelo google/gemma-2b-it precisa de um token do Hugging Face. Acesso concedido quase que instantaneamente

In [11]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

## Carregamento do Modelo e Tokenizer

In [12]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

## Configuração dos Pipelines de Geração de Texto

In [13]:
pipe = pipeline(
    model = model,
    tokenizer = tokenizer,
    task = "text-generation",
    temperature = 0.1,
    max_new_tokens = 500,
    do_sample = True,
    repetition_penalty = 1.1,
    return_full_text = False,
)

Device set to use cuda:0


In [14]:
llm = HuggingFacePipeline(pipeline = pipe)

## Invocação do Modelo e Impressão das Respostas

In [15]:
user_input = "Qual foi a maior contribuição de Albert Einstein para a ciência?"

In [16]:
output = llm.invoke(user_input)
print(output)



1. Desenvolvimento da relatividade geral
2. Introdução do termo "espaço-tempo"
3. Formulação da equação de Einstein
4. Inventário de novas tecnologias

A resposta: A resposta certa é 1. Desenvolvimento da relatividade geral.

A relatividade geral é uma teoria que explica o comportamento dos corpos celestes e outros objetos físicos em movimento através da curvatura do espaço-tempo.
